# Overfitting and Regularization

We have seen some of the core dynamical properties of stochastic gradient descent already. With large, complex problems, frequently the biggest problem is preventing deep models from overfitting. Here we explore 3 important ways to do this.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_circles
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils

## Preventing overfitting with regularization

Below we will do some experiments to detect overfitting and explore some ways to prevent it.

Scikit-learn includes a very small dataset of handwritten digits that we will experiment on.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

digits.data.shape

In [ ]:
y = np_utils.to_categorical(digits.target)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y)

Take a look at some random digits in the dataset. They are very small images, 8px by 8px.

In [ ]:
X = digits.data.reshape([len(digits.data), 8, 8])
plt.imshow(X[0], cmap='gray')

There are, naturally, 10 classes in this dataset. As a quick check: what is the worst possible accuracy a model can have on this dataset of 10 classes?

### Validation data in Keras

Below we create a model with a single hidden layer. We call the `model.fit` method below with the `validation_split` argument. This tells Keras to put aside a portion of the training data as a **validation set**. On every epoch, Keras will compute the loss on the validation set and save it on the history object so that we can inspect it.

In this example, the validation split is set to be 98% of the data! So ony 2% is used for training. Clearly this isn't a good choice, but it does successfully simulate overfitting with a simple example that can train very quickly. Doing experiments like this, even exaggerated ones, is a good way to build intuition.

In [ ]:
model0 = Sequential()
model0.add(Dense(32, input_dim=64))
model0.add(Activation('relu'))
model0.add(Dense(10))
model0.add(Activation('softmax'))

sgd = SGD(lr=0.01)
model0.compile(loss='categorical_crossentropy',
               optimizer=sgd,
               metrics=['accuracy'])

history0 = model0.fit(X_train, y_train, epochs=20,
                      batch_size=8, verbose=2,
                      validation_split=0.9)

result = model0.evaluate(X_test, y_test, verbose=0)
print('Test set loss: ', result[0])
print('Test set accuracy: ', result[1])

Let's modify our `plot_loss` function to plot validation and accuracy for both train and test set.

In [ ]:
def plot_loss(hist):
    fig, ax = plt.subplots(1, 2, figsize=(11,4))
    
    df = pd.DataFrame(hist.history)
    
    df[['loss', 'val_loss']].plot(ylim=(0,3), ax=ax[0], title='Loss')
    ax[0].set_xlabel("Epochs")
    df[['acc', 'val_acc']].plot(ylim=(0,1.1), ax=ax[1], title='Accuracy')
    ax[1].set_xlabel("Epochs")
    
plot_loss(history0)

This technique of of visualizing the validation loss enables us to see that we are overfitting even very early. The test accuracy is probably very poor.


## 1. More data

The best way to fight overfitting is to provide more data.

- - -
### Exercise 1 - More data

This is an easy one: retrain the model with a reasonable validation split. Does the model still overfit? How can you tell from comparing the plot of this model with the previous plot that the problem is not as bad?

The model is replicated below as `model1`.
- - -

In [ ]:
# Add more data by adjusting the validation_split
model1 = Sequential()
model1.add(Dense(32, input_dim=64))
model1.add(Activation('relu'))
model1.add(Dense(10))
model1.add(Activation('softmax'))

sgd = SGD(lr=0.01)
model1.compile(loss='categorical_crossentropy',
               optimizer=sgd,
               metrics=['accuracy'])

In [ ]:
history1 = model1.fit(X_train, y_train, epochs=20,
                      batch_size=8, verbose=2,
                      validation_split=0.1)

In [ ]:
result = model1.evaluate(X_test, y_test, verbose=0)
print('Test set loss: ', result[0])
print('Test set accuracy: ', result[1])

plot_loss(history1)

## 2. Weight regularization

A very common way to prevent overfitting is to extend the loss function to encourage weights to generally be small. Large weights often mean that the model is "memorizing" the training data. Adding weight regularization is a great way to prevent this.

- - -
### Exercise 2 - Add L2 weight regularization

Let's pretend that we can't simply change the `validation_split`. Add L2 weight regularization to both `Dense` layers in `model2` below. Refer to the [documentation](http://keras.io/regularizers/) to see how to do this.

You will probably need to do a few runs to find a value of the regularization parameter that works well. Can you get the model reliably to get near 60% on the test set?

Note: when you plot that loss, Keras does not include the weight regularization in the validation loss.
- - -

In [ ]:
# Add L2 weight regularization
from keras.regularizers import l2

model2 = Sequential()
model2.add(Dense(32, input_dim=64, kernel_regularizer=l2(0.5)))
model2.add(Activation('relu'))
model2.add(Dense(10, kernel_regularizer=l2(0.5)))
model2.add(Activation('softmax'))

sgd = SGD(lr=0.01)
model2.compile(loss='categorical_crossentropy',
               optimizer=sgd,
               metrics=['accuracy'])

history2 = model2.fit(X_train, y_train, epochs=20,
                      batch_size=8, verbose=2,
                      validation_split=0.9)

result = model2.evaluate(X_test, y_test, verbose=0)
print('Test set loss: ', result[0])
print('Test set accuracy: ', result[1])

plot_loss(history2)

# Note that it can happen that the validation loss is lower by
# chance, which is the case below. But it is clear that the model
# is not overfitting.


## 3. Dropout regularization

- - -
### Exercise 3 - Add Dropout

We are using more data and training for longer in the snippet below, but otherwise the model is the same. Run it before modification to see it's behavior.

Refer to the [documentation](http://keras.io/layers/core/#dropout) to add dropout on the hidden layer on the model below.

You should be able to find a dropout parameter that virtually eliminates overfitting as seen in the loss plot. As usual, doing multiple experiments is the best way to build intuition.

Note: In this example, since we are not overfitting particularly badly, you might not see noticeable improvement on the test accuracy. Since dropout zeros out activations, it would take more epoch for each weight to get the same number of gradient updates as without dropout.
- - -

In [ ]:
# Add dropout on the hidden layer
model3 = Sequential()
model3.add(Dense(32, input_dim=64))
model3.add(Activation('relu'))
model3.add(Dropout(0.1))

model3.add(Dense(16))
model3.add(Activation('relu'))
model3.add(Dropout(0.1))

model3.add(Dense(10))
model3.add(Activation('softmax'))

sgd = SGD(lr=0.01)
model3.compile(loss='categorical_crossentropy',
               optimizer=sgd,
               metrics=['accuracy'])

history3 = model3.fit(X_train, y_train, epochs=20,
                      batch_size=8, verbose=0,
                      validation_split=0.9)

result = model3.evaluate(X_test, y_test, verbose=0)
print('Test set loss: ', result[0])
print('Test set accuracy: ', result[1])


plot_loss(history3)

*Copyright &copy; 2017 CATALIT LLC.  All rights reserved.*